# Argument classification with BERT

In [ ]:
import os, sys
from IPython.display import clear_output

In [ ]:
# Load amrlib library from drive
argument_classification_path = '/content/argument_classification'

In [ ]:
! pip install pytorch-pretrained-bert==0.6.2 sklearn scipy
clear_output()

In [ ]:
! pip install flair
from flair.data import Sentence
from flair.models import SequenceTagger
clear_output()

In [ ]:
# load tagger
pos_tagger = SequenceTagger.load("pos-fast")
ner_tagger = SequenceTagger.load("ner-fast")
clear_output()

In [ ]:
def get_sentence_themes(sent):
    # make example sentence
    sentence = Sentence(sent)

    # Create list of sentence themes
    entities = []
    ignore_list = []
    acceptable_ners = ['LOC', 'PER']
    acceptable_pos = ['NNS']

    # predict NER tags
    ner_tagger.predict(sentence)

    # Find the NERs
    for entity in sentence.get_spans('ner'):
        if entity.get_label("ner").value in acceptable_ners:
            entities.append(entity.text)

        # Add tokens to blacklist
        for token in entity:
            ignore_list.append(token.text)

    # predict POS tags
    pos_tagger.predict(sentence)

    # Find the POS
    for token in sentence.tokens:
        if token.get_label("pos").value in acceptable_pos:
            entities.append(token.text)

    entities = list(set(entities))
    if entities == []:
        entities=['']
    print(entities)
    return entities

Create input examples

In [ ]:
from ast import literal_eval
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/IP/datasets/processed_test.csv')

In [ ]:
# Create inputs
os.chdir(argument_classification_path)
from train import InputExample, convert_examples_to_features

all_inputs = []
for idx, art in df.iterrows():
    art_inputs = []
    for sent in eval(art['article_sentences']):
        sent_inputs = []
        entities = get_sentence_themes(sent)
        for entity in entities:
            text_a = entity
            text_b = sent
            label = 'NoArgument'
            example = InputExample(text_a, text_b, label)
            sent_inputs.append(example)
        art_inputs.append([example])
    all_inputs.append(art_inputs)

In [ ]:
import importlib
importlib.reload(inference)

<module 'inference' from '/content/drive/MyDrive/IP/argument_classification/acl2019-BERT-argument-classification-and-clustering/argument-classification/inference.py'>

In [ ]:
import inference
article_scores = []
sent_scores = []

for i, art_inputs in enumerate(all_inputs):
    if i == 1:
        for j, sent_inputs in enumerate(art_inputs):
            topics, sents, preds = inference.predict(sent_inputs)
            verdict = max(set(preds), key=preds.count)
            if verdict == "NoArgument":
                score = 0
            else:
                score = 1
            sent_scores.append(score)
        article_scores.append(sent_scores)

# Memory errors with current setup :(